In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.colors as colors
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
#The data set
house_prices_df = pd.read_csv("pp-2021.csv",
                              usecols=[1,3,4,5],
                              names=["price","postcode","type","new"])
house_prices_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'pp-2021.csv'

In [3]:
#Postcode to Latitude and Longitude
postcodes_df = pd.read_csv("ukpostcodes.csv",usecols=[1,2,3])
postcodes_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'ukpostcodes.csv'

In [4]:

df = house_prices_df.merge(postcodes_df,how='inner',left_on='postcode',right_on='postcode')
df['price'] = df['price'] / 1000
df.head()

NameError: name 'house_prices_df' is not defined

In [ ]:
df['price'].describe()

In [ ]:
#Visualisation
def visualise(df, vmin, vmax):
    
    df_sorted = df.sort_values(by='price')
    x = df_sorted['longitude']
    y = df_sorted['latitude']
    c = df_sorted['price'] 

    plt.rcParams['figure.figsize'] = [5, 6]
    plt.rcParams['figure.dpi'] = 100 

    plt.scatter(x, y, s=0.01, c=c, cmap='plasma_r', 
                norm=colors.Normalize(vmin=vmin,vmax=vmax), alpha=0.8)
    plt.colorbar()
    plt.show()
    
visualise(df, 200, 1000)


In [ ]:
#House Price Prediction by Location
# Terraced houses only (i.e., Townhouses in the US)
prices_df = df[(df['type'] == 'T')]
# Obtain the average price paid in each postcode
prices_df = prices_df.groupby(["postcode","latitude","longitude"], as_index=False)["price"].mean()
prices_df.head()


In [ ]:
#Using k-NN
X = prices_df[['latitude','longitude']]
y = prices_df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)


In [ ]:
uniform  = []
distance = []
r = range (1,21,2)

for k in r:
    
    # Euclidan, 'straight' distance
    model = KNeighborsRegressor(n_neighbors = k, weights='uniform')
    model.fit(X_train.values, y_train.values)
    uniform.append(model.score(X_test.values,y_test.values))

    # Distance is inversely proportional (to lessen the weight of outliers)
    model = KNeighborsRegressor(n_neighbors = k, weights='distance') 
    model.fit(X_train.values, y_train.values)
    distance.append(model.score(X_test.values,y_test.values))

uniform = np.array(uniform)
distance = np.array(distance)

plt.rcParams['figure.figsize'] = [10, 3]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
plt.plot(r,uniform,label='uniform',color='blue')
plt.plot(r,distance,label='distance',color='red')
plt.legend()
plt.gca().set_xticks(r)
plt.show()


In [ ]:
pd.DataFrame({"k" : r, "uniform" : uniform, "distance" : distance})


In [ ]:
params = {'n_neighbors':range(1,21,2),'weights':['uniform','distance']}


In [ ]:
model = GridSearchCV(KNeighborsRegressor(), params, cv=5)
model.fit(X_train.values,y_train.values)
model.best_params_

{'n_neighbors': 19, 'weights': 'uniform'}

In [ ]:
model.score(X_test.values,y_test.values)


In [ ]:
def price(description,lat,lon):
    features = [[lat,lon]]
    print("{:30s} -> {:5.0f}k ".format(description,float(model.predict(features))))

# Examples
price('Oxford Circus, London', 51.515276, -0.142038)
price('Harrods (B. Road), London', 51.499814, -0.163366)
price('Peak District, National Park', 53.328508, -1.783416)


In [ ]:
#House Type by Location and Price

types_df = df.groupby(["latitude","longitude","type"], as_index=False)["price"].mean()
types_df.head()


In [ ]:
params = {'n_neighbors':range(1,21,2),'weights':['uniform','distance']}

X = types_df[['latitude','longitude','price']]
y = types_df['type']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

model = GridSearchCV(KNeighborsClassifier(), params, cv=5)
model.fit(X_train.values,y_train.values)
model.best_params_

{'n_neighbors': 19, 'weights': 'distance'}

In [ ]:
model.score(X_test.values,y_test.values)


In [ ]:
ht = { 'F' : 'Flat', 'T' : 'Terraced', 'S' : 'Semi', 'D' : 'Detached', 'O' : 'Other'}

def house_type(description,lat,lon,price):
    features = [[lat,lon,price]]
    print("{:30s} {:5.0f}k -> {}".format(description,price,ht[(model.predict(features)[0])]))

house_type('Oxford Circus, London', 51.515276, -0.142038, 500)
house_type('Harrods (B. Road), London', 51.499814, -0.163366, 5500)
house_type('Peak District, National Park', 53.328508, -1.783416, 100)
